In [6]:
import pandas as pd 
import numpy as np
df = pd.read_csv("data/output/result.csv", header=None)
df.columns = ['accuracy', 'explanation', 'positive counterexamples', 'query', 'random words checked', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']
print("minimum iterations:", min(df.groupby(['query'])['explanation'].count()), "max iterations:",max(df.groupby(['query'])['explanation'].count()))
df.head()

minimum iterations: 80 max iterations: 80


,accuracy,explanation,positive counterexamples,query,random words checked,terminate,time,time learner,time verifier,total counterexamples
0,1.0,(not (or eggs breathes)),0.4,breathes = 0,247.0,1.0,26.672838,1.451271,25.218679,5.0
1,1.0,(not (or eggs breathes)),0.5,breathes = 0,263.0,1.0,28.461545,1.688770,26.769444,6.0
2,1.0,(not (or eggs breathes)),0.5,breathes = 0,266.0,1.0,28.712535,1.734702,26.974814,6.0
3,1.0,(not (or eggs breathes)),0.4,breathes = 0,268.0,1.0,28.708704,1.406770,27.299391,5.0
4,1.0,(not (or eggs breathes)),0.5,breathes = 0,269.0,1.0,29.129722,1.708946,27.417850,6.0


In [7]:
# take median
group_list = ['query']
df_med = df.groupby(group_list).median()
df_med.reset_index(inplace=True)
df_med.head()

,query,accuracy,positive counterexamples,random words checked,terminate,time,time learner,time verifier,total counterexamples
0,aquatic = 0,0.727273,0.466667,226.5,0.0,401.103211,375.705307,24.330269,15.0
1,backbone = 1,0.727273,0.466667,277.0,0.0,401.103397,372.452096,27.630340,16.0
2,breathes = 0,1.000000,0.428571,288.5,1.0,11.357691,0.502495,10.884681,7.0
3,eggs = 0,0.909091,0.470588,296.5,0.0,401.102876,365.618165,34.446124,15.0
4,legs <= 0.2,0.727273,0.466667,308.5,0.0,401.103456,363.788377,36.311102,14.0


In [8]:
# find representative explanation
query_explanations = {}
for key, item in df.groupby(['query'], as_index=False):
    item.reset_index(inplace=True, drop=True)
    # nearest explanation to the average accuracy
    explanation = item.iloc[item.index[(item['accuracy']-df_med['accuracy'][(df_med['query'] == key)].item()).abs().argsort()][0]]['explanation']
    query_explanations[key] = explanation

In [9]:
df_output = df_med
df_output = df_output.drop("terminate", axis=1)
df_output['explanation'] = np.nan 
for key in query_explanations:
    df_output['explanation'][df_output['query'] == key] = query_explanations[key]

# reorganise columns
df_output = df_output[['query', 'explanation', 'accuracy', 'time', 'time learner', 'time verifier', 'random words checked', 'total counterexamples', 'positive counterexamples']]
df_output = df_output.round(2)
df_output.head()

,query,explanation,accuracy,time,time learner,time verifier,random words checked,total counterexamples,positive counterexamples
0,aquatic = 0,(not (or eggs (or aquatic (and breathes (not ...,0.73,401.10,375.71,24.33,226.5,15.0,0.47
1,backbone = 1,(and backbone (not (and (or eggs toothed) (or...,0.73,401.10,372.45,27.63,277.0,16.0,0.47
2,breathes = 0,(not (or eggs breathes)),1.00,11.36,0.50,10.88,288.5,7.0,0.43
3,eggs = 0,(not (or eggs (and breathes (not (and airborn...,0.91,401.10,365.62,34.45,296.5,15.0,0.47
4,legs <= 0.2,(or (and hair venomous) (or (and eggs milk) (...,0.73,401.10,363.79,36.31,308.5,14.0,0.47


In [10]:
# to remote 
# df_output.to_csv("data/output/summary.csv", index = False)
df_output.to_csv("/home/bishwamittra/Dropbox/trustable_explanations/result.csv", index = False)